## traditional

In [1]:
import numpy as np

def make_cluster(natoms, radius=20, seed=1981):
    np.random.seed(seed)
    arr = np.random.normal(0, radius, size=(natoms,3))-0.5
    return arr

In [2]:
class lj_pure(object):
    
    @classmethod
    def lj(cls, r):
        sr6 = (1./r)**6
        pot = 4.*(sr6*sr6 - sr6)
        return pot


    @classmethod
    def distance(cls, atom1, atom2):
        dx = atom2[0] - atom1[0]
        dy = atom2[1] - atom1[1]
        dz = atom2[2] - atom1[2]

        r = (dx*dx + dy*dy + dz*dz)**0.5
        return r


    @classmethod
    def potential(cls, cluster):
        energy = 0.0
        for i in range(len(cluster)-1):
            for j in range(i+1,len(cluster)):
                r = cls.distance(cluster[i],cluster[j])
                e = cls.lj(r)
                energy += e
        return energy

In [3]:
import numpy as np
class lj_numpy(object):
    
    @classmethod
    def lj(cls, r):
        sr6 = (1./r)**6
        pot = 4.*(sr6*sr6 - sr6)
        return pot
    
    
    @classmethod
    def distances(cls, cluster):
        diff = cluster[:, np.newaxis, :] - cluster[np.newaxis, :, :]
        mat = np.sqrt((diff*diff).sum(-1))
        return mat

    
    @classmethod
    def potential(cls, cluster):
        d = cls.distances(cluster)
        dtri = np.triu(d)
        energy = cls.lj(dtri[dtri > 1e-4]).sum()
        return energy

In [4]:
cluster = make_cluster(int(2e3), radius=100)

In [5]:
%timeit lj_pure.potential(cluster)

4.44 s ± 81.3 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [6]:
%timeit lj_numpy.potential(cluster)

297 ms ± 889 µs per loop (mean ± std. dev. of 7 runs, 1 loop each)


## cython

In [7]:
%%writefile tmp/setup.py
from setuptools import setup, Extension
from Cython.Build import cythonize
import numpy

setup(
    name = "lennard-jones library",
    ext_modules = cythonize([
        Extension("lj", ['tmp/lj.pyx'],
                  extra_compile_args = ["-Wno-unused-function", '-Wno-cpp', '-O2', '-fopenmp'],
                  extra_link_args = ['-fopenmp']
                 )
    ]),
    include_dirs = [numpy.get_include()]
)

Overwriting tmp/setup.py


In [13]:
%%writefile tmp/lj.pyx

cimport cython
cimport numpy as np
import numpy as np
ctypedef np.float_t DOUBLE
from cython.parallel import prange

@cython.boundscheck(False)
def potential(np.ndarray[DOUBLE, ndim=2] cluster):
    cdef DOUBLE energy = 0.0
    cdef int n_atoms = cluster.shape[0]
    cdef DOUBLE e, r, dx, dy, dz, sr6
    cdef int i,j
    cdef int ZERO = 0
    cdef int ONE = 1
    cdef int TWO = 2
    for i in range(n_atoms-1):
        for j in prange(i+1,n_atoms, nogil=True):
            #r = distance(cluster[i],cluster[j])
            #e = lj(r)
            
            dx = cluster[j, ZERO] - cluster[i, ZERO]
            dy = cluster[j, ONE] - cluster[i, ONE]
            dz = cluster[j, TWO] - cluster[i, TWO]
            r = (dx*dx + dy*dy + dz*dz)**0.5
            
            sr6 = (1./r)**6
            e = 4.*(sr6*sr6 - sr6)
            
            energy += e
    return energy

Overwriting tmp/lj.pyx


In [14]:
import sys
!CC=gcc-mp-7 {sys.executable} tmp/setup.py build_ext --inplace

Compiling tmp/lj.pyx because it changed.
[1/1] Cythonizing tmp/lj.pyx
running build_ext
building 'lj' extension
gcc-mp-7 -Wno-unused-result -Wsign-compare -Wunreachable-code -DNDEBUG -g -fwrapv -O3 -Wall -Wstrict-prototypes -I/Users/albert/Applications/miniconda3/envs/intel/include -arch x86_64 -I/Users/albert/Applications/miniconda3/envs/intel/include -arch x86_64 -I/Users/albert/Applications/miniconda3/envs/intel/lib/python3.6/site-packages/numpy/core/include -I/Users/albert/Applications/miniconda3/envs/intel/include/python3.6m -c tmp/lj.c -o build/temp.macosx-10.6-x86_64-3.6/tmp/lj.o -Wno-unused-function -Wno-cpp -O2 -fopenmp
gcc-mp-7 -bundle -undefined dynamic_lookup -L/Users/albert/Applications/miniconda3/envs/intel/lib -L/Users/albert/Applications/miniconda3/envs/intel/lib -arch x86_64 build/temp.macosx-10.6-x86_64-3.6/tmp/lj.o -L/Users/albert/Applications/miniconda3/envs/intel/lib -o build/lib.macosx-10.6-x86_64-3.6/lj.cpython-36m-darwin.so -fopenmp
copying build/lib.macosx-10.6

In [16]:
#import lj
from importlib import reload
lj = reload(lj)

%timeit lj.potential(cluster)

208 ms ± 15.2 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [ ]:
potential(cluster)